## My little database JSON-LD import

#### Import relevant packages

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing
import bw_migrations

import os
import numpy as np
import pandas as pd

#### Create/set the working folder:

In [2]:
bd.projects.dir

PosixPath('/Users/mmendez/Library/Application Support/Brightway3/default.c21f969b5f03d33d43e04f8f136e7682')

In [3]:
bd.projects.set_current('MLD_JSON_0')

#### Import JSON-LD:

In [6]:
path = '/Users/mmendez/Documents/Postdoc/Software_dev/Brightway/Brightway_US/US_LCI_BW_Project/notebooks/databases/My_little_database/JSON-LD/My_little_database'
mld = bi.importers.JSONLDImporter(
    path, 
    "My_little_dataset_v0", 
    preferred_allocation="PHYSICAL_ALLOCATION"
)

#### Apply strategies to map JSON-LD to Brightway2 schema:

In [6]:
mld.apply_strategies()

Applying strategy: json_ld_allocate_datasets
Applying strategy: json_ld_get_normalized_exchange_locations
Applying strategy: json_ld_convert_unit_to_reference_unit
Applying strategy: json_ld_get_activities_list_from_rawdata
Applying strategy: json_ld_add_products_as_activities
Applying strategy: json_ld_get_normalized_exchange_units
Applying strategy: json_ld_add_activity_unit
Applying strategy: json_ld_rename_metadata_fields
Applying strategy: json_ld_location_name
Applying strategy: json_ld_remove_fields
Applying strategy: json_ld_fix_process_type
Applying strategy: json_ld_label_exchange_type
Applying strategy: json_ld_prepare_exchange_fields_for_linking
Applying strategy: add_database_name
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: normalize_units
Applied 17 strategies in 0.00 seconds

	Created 3 biosphere flows in separate database 'My_little_dataset_v0 biosphere'.
	Use either `.merge_biosphere_flows()` or `.write_separ

#### Check database dictionaries:

In [7]:
bd.databases # As expected, nothing was written.

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

#### Write the biosphere database:

In [8]:
mld.write_separate_biosphere_database()

Writing activities to SQLite3 database:
0% [###] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/14/2022 09:44:55
  Finished: 09/14/2022 09:44:55
  Total time elapsed: 00:00:00
  CPU %: 2.10
  Memory %: 0.50
Created database: My_little_dataset_v0 biosphere


In [9]:
bd.databases

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

#### Write the technosphere database:

In [10]:
bio = bd.Database('My_little_dataset_v0 biosphere')

In [11]:
mld.write_database()

Writing activities to SQLite3 database:


Not able to determine geocollections for all datasets. This database is not ready for regionalization.


0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 09/14/2022 09:44:56
  Finished: 09/14/2022 09:44:56
  Total time elapsed: 00:00:00
  CPU %: 2.60
  Memory %: 0.51
Created database: My_little_dataset_v0


Brightway2 SQLiteBackend: My_little_dataset_v0

<div class="alert alert-block alert-warning">
<b>To fix:</b>
    <li>The importer does not recognize the geolocations.</li>
    <li>The importer does not import the LCIA methods. <i>Hint:</i> Check bw2setup() function, this one loads LCIAs and biosphere flows.</li>
</div>

In [12]:
bd.databases

Databases dictionary with 2 object(s):
	My_little_dataset_v0
	My_little_dataset_v0 biosphere

In [13]:
db = bd.Database('My_little_dataset_v0')

#### Let's look at the biosphere flows:

In [14]:
[act for act in bio]

['Bad stuff' (, None, ('Elementary flows', 'emissions')),
 'Virgin metals' (, None, ('Elementary flows', 'resources')),
 'Crude oil' (, None, ('Elementary flows', 'resources'))]

#### And now let's look at the our technosphere processes and products:

In [15]:
[act for act in db]

['Plastic manufacturer' (kilogram, Djibouti, None),
 'Stainless steel' (, Djibouti, ('Technosphere flows', 'Manufacturers')),
 'Stainless steel manufacturer' (kilogram, Djibouti, None),
 'Bottle' (, Djibouti, ('Technosphere flows', 'Assemblers')),
 'Bottle assembly' (number_of_items, Djibouti, None),
 'Plastic' (, Djibouti, ('Technosphere flows', 'Manufacturers'))]

#### Which ones are processes and which ones are products? Let's also get their codes:

In [16]:
[(act['name'], act['type'], act['code']) for act in db]

[('Stainless steel', 'product', '3355a1b9-8fbf-40de-b449-ea6399a8a323'),
 ('Bottle', 'product', 'b806c2cd-d563-43c4-a0c9-9c7dd5d513d3'),
 ('Bottle assembly', 'process', 'fae6799b-7326-452c-92b3-76758bbcac22'),
 ('Plastic', 'product', '62714200-1a0d-43fb-9b48-99df3f233c94'),
 ('Plastic manufacturer', 'process', '2fc6deea-6437-4b9f-bf91-89bca44d30f0'),
 ('Stainless steel manufacturer',
  'process',
  '7e5ec332-09fd-4706-8373-3f140a539028')]

### Let's manually add a LCIA method
This importer does not recognize the LCIA methods, therefore we need to add them manually. I will debug this later!

First, let's find the `Bad stuff` biosphere flow.

In [17]:
bad_flow = [act for act in bio if act['name'] == 'Bad stuff'][0]
bad_flow.as_dict()

{'code': 'cd154d8f-0694-43b2-b4ab-e44101e122bd',
 'name': 'Bad stuff',
 'categories': ('Elementary flows', 'emissions'),
 'CAS number': None,
 'database': 'My_little_dataset_v0 biosphere',
 'unit': '',
 'type': 'emission',
 'id': 12}

In [18]:
myLCIAdata = [[(bad_flow['database'], bad_flow['code']), 2.0]] # A method list needs: a reference to the flow: tuple (database, 'code')), a characterization factor number, and localization (if no localization is given, 'GLO' is used)
method_key = ('MacIM', 'Global warming', 'total')
my_method = bd.Method(method_key)
my_method.validate(myLCIAdata)
my_method.register()
my_method.write(myLCIAdata)

#### Now we define a functional unit:
This one might be a bit counterintuitive, our functional unit here is **Impact of assembling 5 bottles**, intuintively one would select the activity, but bw2 selects the flow coming out of the `Bottle assembly` activity (i.e. `Bottle`, which is a `product` not a `process`).

In [19]:
activity = [act for act in db if act['name'] == 'Bottle'][0]

In [20]:
functional_unit = {activity : 5}

#### Run the LCA!

In [21]:
lca = bc.LCA(functional_unit, method_key) 
lca.lci() 
lca.lcia()
lca.score
print(lca.inventory)

  (0, 1)	1.5000000596046448
  (1, 0)	1.25
  (2, 2)	0.5000000074505806
  (2, 1)	4.500000178813934
  (2, 0)	2.5


In [22]:
lca.score

15.00000037252903

In [23]:
mld.data

[{'name': 'Plastic manufacturer',
  'description': '',
  'version': '00.00.005',
  'defaultAllocationMethod': 'PHYSICAL_ALLOCATION',
  'location': 'Djibouti',
  'processDocumentation': {'@type': 'ProcessDocumentation',
   'copyright': False,
   'creationDate': '2022-08-29T17:47:32.435-06:00'},
  'lastInternalId': 3,
  'exchanges': [{'amount': 1.0,
    'unit': 'kilogram',
    'type': 'production',
    'name': 'Plastic',
    'code': '62714200-1a0d-43fb-9b48-99df3f233c94',
    'input': ('My_little_dataset_v0', '62714200-1a0d-43fb-9b48-99df3f233c94'),
    'output': ('My_little_dataset_v0',
     '2fc6deea-6437-4b9f-bf91-89bca44d30f0')},
   {'amount': 2.0,
    'unit': 'kilogram',
    'type': 'biosphere',
    'name': 'Bad stuff',
    'code': 'cd154d8f-0694-43b2-b4ab-e44101e122bd',
    'input': ('My_little_dataset_v0 biosphere',
     'cd154d8f-0694-43b2-b4ab-e44101e122bd'),
    'output': ('My_little_dataset_v0',
     '2fc6deea-6437-4b9f-bf91-89bca44d30f0')},
   {'amount': 1.0,
    'unit': 'kil